In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb
import glob

In [2]:
#Getting the api login info
with open('/Users/alicechen/.secret/tmdb_api.json') as f:
    login = json.load(f)
login.keys()
tmdb.API_KEY = login['API Key']

In [3]:
def get_movie_with_rating(movie_id):
    #Get the movies object for current id 
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    #Loop through countries in releases
    for c in  releases['countries']:
        if c['iso_3166_1'] == 'US':
        # if the country == US
        ## GET "certification" Key
            info["certification"]= c["certification"]
    return info

In [4]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'title_rating.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'final_tmdb_data_2013.csv.gz',
 'title_akas.csv.gz',
 'tmdb_results_combined.csv.gz']

In [5]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [6]:
YEARS_TO_GET = [2020,2021]
errors = [ ]

In [7]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [8]:
# start of the OUTER LOOP
for YEAR in tqdm_notebook(YEARS_TO_GET, desc= 'YEARS', position = 0):

    # define the JSON file to store results
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # check if the file exists
    file_exists = os.path.isfile(JSON_FILE)

    # if the file doesn't exist, create it
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)

    # save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()

    # save movie ids to list
    movie_ids = df['tconst'].copy() #to_list()
    movie_ids

    # load existing data fro json into new file
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position = 1,
                                 leave = True):
        try:
            temp = get_movie_with_rating(movie_id)
            write_json(temp,JSON_FILE)
            time.sleep(0.02)
        except Exception as e: 
            errors.append([movie_id, e])
            

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2020:   0%|          | 0/5215 [00:00<?, ?it/s]

Movies from 2021:   0%|          | 0/5309 [00:00<?, ?it/s]

In [11]:
print(f"- Total errors: {len(errors)}")

- Total errors: 2347


In [15]:
#Use glob to append data
q = "Data/final_tmdb_data_*.csv.gz"
chunked_files = glob.glob(q)
chunked_files

['Data/final_tmdb_data_2018.csv.gz',
 'Data/final_tmdb_data_2014.csv.gz',
 'Data/final_tmdb_data_2016.csv.gz',
 'Data/final_tmdb_data_2020.csv.gz',
 'Data/final_tmdb_data_2000.csv.gz',
 'Data/final_tmdb_data_2012.csv.gz',
 'Data/final_tmdb_data_2019.csv.gz',
 'Data/final_tmdb_data_2015.csv.gz',
 'Data/final_tmdb_data_2021.csv.gz',
 'Data/final_tmdb_data_2017.csv.gz',
 'Data/final_tmdb_data_2001.csv.gz',
 'Data/final_tmdb_data_2013.csv.gz']

In [25]:
## Loading all files as df and appending to a list
df_list = []
for file in chunked_files:
    print(file)
    temp_df = pd.read_csv(file,usecols=['imdb_id','revenue','budget','certification'])
    df_list.append(temp_df)
    tmdb_data = pd.concat(df_list)

Data/final_tmdb_data_2018.csv.gz
Data/final_tmdb_data_2014.csv.gz
Data/final_tmdb_data_2016.csv.gz
Data/final_tmdb_data_2020.csv.gz


ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [33]:
tmdb_data

,imdb_id,budget,revenue,certification
1,tt0069049,12000000.0,0.0,R
2,tt0192528,5000000.0,0.0,NaN
3,tt0360556,0.0,0.0,PG-13
4,tt0365545,0.0,0.0,NaN
5,tt0427543,0.0,0.0,NaN
...,...,...,...,...
3954,tt9387300,0.0,0.0,NaN
3955,tt9434008,0.0,0.0,NaN
3956,tt9547878,0.0,0.0,NaN
3957,tt9741588,0.0,0.0,NaN


In [31]:
tmdb_data = tmdb_data[tmdb_data.imdb_id != '0']